This project demonstrates how to build and execute parallel workflows using LangGraph. It showcases concurrent task execution, state management, and orchestration, highlighting how LangGraph enables efficient handling of multiple processes simultaneously.

In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# Annotated: Annotated is a special typing construct that allows you to add metadata to types.
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

load_dotenv()

True

In [2]:
# define model, gpt-4o-mini as it supports Pydantic, strcutured output
model = ChatOpenAI(model='gpt-4o-mini')

# Define the schema for the evaluation output
class EvaluationSchema(BaseModel):

    # providing description helps LLM understand the context better
    feedback: str = Field(description='Detailed feedbackfor the essay')

    # greater than or equal to 0, less than or equal to 10
    score: int = Field(description='Score out of 10', ge=0, le=10)

structured_model = model.with_structured_output(EvaluationSchema)

#### Example of structured output

In [3]:
essay = """India in the Age of AI
As the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.

India's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for All) with a focus on inclusive growth, aiming to leverage AI in healthcare, agriculture, education, and smart mobility.

One of the most promising applications of AI in India lies in agriculture, where predictive analytics can guide farmers on optimal sowing times, weather forecasts, and pest control. In healthcare, AI-powered diagnostics can help address India’s doctor-patient ratio crisis, particularly in rural areas. Educational platforms are increasingly using AI to personalize learning paths, while smart governance tools are helping improve public service delivery and fraud detection.

However, the path to AI-led growth is riddled with challenges. Chief among them is the digital divide. While metropolitan cities may embrace AI-driven solutions, rural India continues to struggle with basic internet access and digital literacy. The risk of job displacement due to automation also looms large, especially for low-skilled workers. Without effective skilling and re-skilling programs, AI could exacerbate existing socio-economic inequalities.

Another pressing concern is data privacy and ethics. As AI systems rely heavily on vast datasets, ensuring that personal data is used transparently and responsibly becomes vital. India is still shaping its data protection laws, and in the absence of a strong regulatory framework, AI systems may risk misuse or bias.

To harness AI responsibly, India must adopt a multi-stakeholder approach involving the government, academia, industry, and civil society. Policies should promote open datasets, encourage responsible innovation, and ensure ethical AI practices. There is also a need for international collaboration, particularly with countries leading in AI research, to gain strategic advantage and ensure interoperability in global systems.

India’s demographic dividend, when paired with responsible AI adoption, can unlock massive economic growth, improve governance, and uplift marginalized communities. But this vision will only materialize if AI is seen not merely as a tool for automation, but as an enabler of human-centered development.

In conclusion, India in the age of AI is a story in the making — one of opportunity, responsibility, and transformation. The decisions we make today will not just determine India’s AI trajectory, but also its future as an inclusive, equitable, and innovation-driven society."""

In [4]:
# output is EvaluationSchema which contains feedback and score as keys

prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
structured_model.invoke(prompt)

EvaluationSchema(feedback='The essay provides a well-structured and insightful analysis of India\'s position regarding artificial intelligence. The language used is formal and appropriate for an academic or professional audience, with a clear progression of ideas from strengths to challenges. The use of examples and statistics enhances the credibility of the arguments. However, there are some areas for improvement:\n1. **Clarity and Conciseness**: While the essay covers a lot of ground, there are sections where sentences could be more concise to improve readability. For example, rather than stating "AI could exacerbate existing socio-economic inequalities," simplifying this to "AI may increase socio-economic inequalities" could enhance clarity.\n2. **Transitions**: Some transitions between paragraphs could be smoother to help guide the reader through the various sections of the argument. For instance, while the shift from strengths to challenges is necessary, a transitional sentence co

#### Let's create Graph

In [5]:
class EssayEvaluator(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str

    # Annotated[base_type, metadata1, metadata2, ...]
    # individual_scores is a list of integers, with additional metadata 
    # saying it’s associated with the operator.add function.
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [ ]:
# node 1 to evaluate language
# partial updates, we don't need to return full state, we can partial update
# and we need to return as dictionary instead of full state object
# for parallel nodes, we don't need to update all State vars everytime

def evaluate_language(state: EssayEvaluator):
    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'language_feedback': output.feedback, 'individual_scores': [output.score]}

# node 2 to evaluate analysis
def evaluate_analysis(state: EssayEvaluator):

    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'analysis_feedback': output.feedback, 'individual_scores': [output.score]}

# node 3 to evaluate clarity of thought
def evaluate_thought(state: EssayEvaluator):

    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'clarity_feedback': output.feedback, 'individual_scores': [output.score]}

# final node to provide overall feedback and avg score
def final_evaluation(state: EssayEvaluator):

    # summary feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'

    # note, here using model instead of structured_model as output is simple string
    overall_feedback = model.invoke(prompt).content

    # avg calculate
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}
    

In [7]:
# let's make Graph
graph = StateGraph(EssayEvaluator)

# nodes
graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)

# edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

# compile
workflow = graph.compile()

In [14]:
# visualize
try:
    from IPython.display import Image
    Image(workflow.get_graph().draw_mermaid_png())
except:
    pass

In [15]:
# we can visualize the graph using mermaid syntax
# https://mermaid.live/
# sometime due to network issues, image may not render, in that case use mermaid_code below to visualize

mermaid_code = workflow.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	evaluate_language(evaluate_language)
	evaluate_analysis(evaluate_analysis)
	evaluate_thought(evaluate_thought)
	final_evaluation(final_evaluation)
	__end__([<p>__end__</p>]):::last
	__start__ --> evaluate_analysis;
	__start__ --> evaluate_language;
	__start__ --> evaluate_thought;
	evaluate_analysis --> final_evaluation;
	evaluate_language --> final_evaluation;
	evaluate_thought --> final_evaluation;
	final_evaluation --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [9]:
# initial state
intial_state = {
    'essay': essay
}

# run graph
final_state = workflow.invoke(intial_state)

In [10]:
final_state

{'essay': "India in the Age of AI\nAs the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.\n\nIndia's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for 